# Golomb Self Describing Sequence
The Golomb's self-describing sequence $G(n)$ is the only nondecreasing sequence of natural numbers such that $n$ appears exactly $G(n)$ times in the sequence. The first few terms of the sequence $n$ are:
$
1, 2, 2, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 6
$

You are given $G(10^3) = 86$ and $G(10^6) = 6137$. 

You are also give $\sum_{i=1}^{10^3} G(i^3) = 153506976$. 

Find $\sum_{i=1}^{10^6} G(i^3)$. 

In [ ]:
def generate_n_golomb(n):
    out = [1, 2]
    curr = 2
    used = 1
    idx = 2

    while idx < n:
        if used < out[curr-1]:
            out.append(curr)
            used += 1
        else:
            curr += 1
            used = 1
            out.append(curr)
        idx += 1
    return out[-1]

In [36]:
from functools import lru_cache

def rec_nth_golomb(n):
    @lru_cache(int(10e6))
    def rec(n):
        if n == 1: 
            return 1
        return 1 + rec(n -rec(rec(n-1)))
    return rec(n)

rec_nth_golomb(100)

21

In [42]:
def golomb_dp(n):
    n = int(n)
    if n == 1:
        return 1

    # Create a list to store Golomb sequence values
    golomb_seq = [0] * (n + 1)
    golomb_seq[1] = 1

    for i in range(2, n + 1):
        golomb_seq[i] = 1 + golomb_seq[i - golomb_seq[golomb_seq[i - 1]]]

    return golomb_seq[n]

In [13]:
import timeit

def timed_g():
    return generate_n_golomb(1e4)

timeit.timeit(timed_g, number=1000)/1000

0.000581604872000753

In [37]:
generate_n_golomb(5)

[1, 2, 2, 3, 3]

In [40]:
a = generate_n_golomb(1e6+1)

In [41]:
a[int(1e6)]

6137

In [28]:
golden = (1 + 5 ** 0.5) / 2

In [43]:
golomb_dp(10e7)

TypeError: can't multiply sequence by non-int of type 'float'